# Decision Trees
Decision Trees are ML algorithms that progressively divide data sets into smaller data groups based on a descriptive feature, until they reach sets that are small enough to be described by some label.

### Main DT Algorithms
####  1. CHAID - Chi-squared Automatic Interaction Detection
When building **classification trees**, CHAID relies on chi-squared tests to find the best split at each step. In other words, it chooses the independent variable that has the strongest interaction with the dependent variable. For **regression trees**, CHAID relies on F-tests to calculate the difference between two population means.

#### 2. CART - Classification And Regression Trees
In the case of **Classification Trees**, CART algorithm uses a metric called **Gini Impurity** to create decision points for classification tasks. Gini Impurity gives an idea of how fine a split is. In the case of **Regression Trees**, CART algorithm looks for splits that minimize the **Least Square Deviation (LSD)**, choosing the partitions that minimize the result over all possible options. The LSD (sometimes referred as “variance reduction”) metric minimizes the sum of the squared distances (or deviations) between the observed values and the predicted values.

#### 3. ID3 - Iterative Dichotomiser 3
It is mostly used for classification tasks. ID3 splits data attributes (dichotomizes) to find the most dominant features, performing this process iteratively to select the DT nodes in a top-down approach. For the splitting process, ID3 uses the **Information Gain** metric to select the most useful attributes for classification. Information Gain is directly linked to the concept of **Entropy**, which is the measure of the amount of uncertainty or randomness in the data.

#### 4. C 4.5 
It is successor of ID3. C4.5 can handle both continuous and categorical data, making it suitable to generate Regression and Classification Trees. Additionally, it can deal with missing values by ignoring instances that include non-existing data. Unlike ID3, C4.5 uses **Gain Ratio** for its splitting process. Gain Ratio is a modification of the Information Gain concept that reduces the bias on DTs. Another capability of C4.5 is that it can prune DTs.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Here we will implement CART algorithm for Regression task
#### Important points
1. The representation of the CART model is a binary tree.
2. For regression, The cost function that is minimized to choose split points is the sum squared error(MSE) across all training samples that fall within the node.
3. For classification, The Gini cost function is used which provides an indication of how pure the nodes are, where node purity refers to how mixed the training data assigned to each node is.

### 1. MSE Criterion
* Name of the cost function used to evaluate splits in the dataset.
* Performs only binary splits
* Higher the value of mse higher the error.

**Steps to calculate cost**:
1. Calculate Mean squared Error of y for each node.
2. Calculate Gini for split using weighted mse of each node of that split


$$ Cost\;for\;split = \sum ( MSE\;for\;node\;y\;values * \frac{node\;size}{total\;size}) $$



### 2. Terminal Node
When to decide to stop growing the tree

**Maximum Tree Depth** : This is the maximum number of nodes from the root node of the tree. Once a maximum depth of the tree is met, we must stop splitting adding new nodes. Deeper trees are more complex and are more likely to overfit the training data.

**Minimum Node Records** :  This is the minimum number of training patterns that a given node is responsible for. Once at or below this minimum, we must stop splitting and adding new nodes. Nodes that account for too few training patterns are expected to be too specific and are likely to overfit the training data.

There is one more condition. It is possible to choose a split in which all rows belong to one group. In this case, we will be unable to continue splitting and adding child nodes as we will have no records to split on one side or another.

Now we have some ideas of when to stop growing the tree. When we do stop growing at a given point, that node is called a terminal node and is used to make a final prediction.

This is done by taking the group of rows assigned to that node and selecting the most common class value in the group. This will be used to make predictions.

In [2]:
class DecisionTreeRegressor():
    def __init__(self,criterion='mse'):
        """
        Args :
        criterion : mse : mean squared error, mae : mean absoulute error, std : standard deviation
        """
        self.root = None
        self.max_depth = 0
        self.cost = { 'mse':self.__mse,'std':self.__std,'mae':self.__mae }[criterion]
        self.X = None
        self.y = None
        
    def __std(self,y):
        squared_error = (y-y.mean())**2
        return np.sqrt(  np.sum(squared_error)/len(y)  )
    
    def __mse(self,y):
        squared_error = (y-y.mean())**2
        return np.sum( squared_error/len(y) )
    
    def __mae(self,y): return np.sum(abs(y-y.mean())/len(y))
    
    def computeCost(self,groups, y):
        n_instances = len(groups[0])+len(groups[1])  # count of all samples
        weighted_cost = 0.0 # sum weighted Gini index for each group
        for indexes in groups:
            size = len(indexes)
            # avoid divide by zero
            if size == 0: continue
            weighted_cost +=  self.cost(y[indexes]) * (size/n_instances)
        return weighted_cost
    
    def get_split(self,X,y):
        b_index, b_value, b_cost, b_groups = float('inf'), float('inf'), float('inf'), None
        for col_ind in range(X.shape[1]): #no of features
            for val in np.unique(X[:,col_ind]): #for each unique value in each of the features

                #left_index indexes lower than val for feature, right_index indexes greater that val for feature
                left_index = np.reshape( np.argwhere(X[:,col_ind]<val), (-1,) )
                right_index = np.reshape( np.argwhere(X[:,col_ind]>=val), (-1,) )
                
                #find gini index
                cost = self.computeCost((left_index,right_index), y)
                if cost < b_cost:
                    b_index, b_value, b_cost, b_groups = col_ind, val, cost, (left_index, right_index)
        return {'index':b_index, 'value':b_value, 'groups':b_groups}
    
    def to_terminal(self,y): return y.mean()
    
    def split(self,node, X, y, max_depth, min_samples_split, depth):
        self.max_depth = max(depth,self.max_depth)
        left, right = node.pop('groups')
        
        # check for a no split
        if len(left)==0 or len(right)==0:
            node['left'] = node['right'] = self.to_terminal(y[np.append(left,right)])
            return
        
        # check for max depth
        if depth >= max_depth:
            node['left'], node['right'] = self.to_terminal(y[left]), self.to_terminal(y[right])
            return
        
        # process left child
        if len(left) <= min_samples_split:
            node['left'] = self.to_terminal(y[left])
        else:
            node['left'] = self.get_split(X[left],y[left])
            self.split(node['left'], X[left], y[left], max_depth, min_samples_split, depth+1)
        
        # process right child
        if len(right) <= min_samples_split:
            node['right'] = self.to_terminal(y[right])
        else:
            node['right'] = self.get_split(X[right],y[right])
            self.split(node['right'],X[right],y[right], max_depth, min_samples_split, depth+1)

    def fit(self, X, y, max_depth=None, min_samples_split=2):
        self.X, self.y, max_depth = X, y, float('inf') if max_depth==None else max_depth
        self.root = self.get_split(X,y)
        self.split(self.root, X, y, max_depth, min_samples_split,1)
        
    def predict_row(self,row,node):
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict): return self.predict_row(row,node['left'])
            else: return node['left']
        else:
            if isinstance(node['right'], dict): return self.predict_row(row,node['right'])
            else: return node['right']
    
    def predict(self,rows): return np.array( [self.predict_row(row,self.root) for row in rows] )
    
    def score(self,X,y):
        "Coefficient of Determination: r2"
        y_pred = self.predict(X)
        return 1-( np.sum( (y-y_pred)**2 )/np.sum( (y-y.mean())**2 ) )
    
    @property
    def depth(self): return self.max_depth
    
    @property
    def tree_(self): return self.root

In [3]:
data = pd.read_csv('data.csv')
data = data.sample(frac=1)
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
288,0.04590,52.5,5.32,0,0.405,6.315,45.6,7.3172,6,293,16.6,396.90,7.60,22.3
315,0.25356,0.0,9.90,0,0.544,5.705,77.7,3.9450,4,304,18.4,396.42,11.50,16.2
249,0.19073,22.0,5.86,0,0.431,6.718,17.5,7.8265,7,330,19.1,393.74,6.56,26.2
307,0.04932,33.0,2.18,0,0.472,6.849,70.3,3.1827,7,222,18.4,396.90,7.53,28.2
469,13.07510,0.0,18.10,0,0.580,5.713,56.7,2.8237,24,666,20.2,396.90,14.76,20.1


In [4]:
def train_test_split(X,y,test_size=0.3):
    indexes = np.random.choice( [False,True], len(y), p=[test_size,1-test_size])
    return X[indexes],X[~indexes],y[indexes],y[~indexes]

X = data.drop('medv',axis=1).values
y = data.medv.values

X_train,X_val,Y_train,Y_val = train_test_split(X,y)
X_train.shape,X_val.shape,Y_train.shape,Y_val.shape

((364, 13), (142, 13), (364,), (142,))

In [5]:
dt = DecisionTreeRegressor(criterion='mse')
dt.fit(X_train,Y_train)
dt.score(X_val,Y_val),dt.depth

(0.7425657291403263, 20)

In [6]:
np.unique(dt.predict(X_val),return_counts=True)

(array([ 7.85,  8.3 ,  8.45,  8.8 ,  9.7 , 10.35, 10.5 , 11.4 , 12.3 ,
        12.9 , 13.3 , 13.35, 13.4 , 13.6 , 13.95, 14.45, 14.5 , 14.55,
        14.85, 15.6 , 16.05, 16.8 , 17.1 , 17.25, 17.4 , 17.55, 18.25,
        18.3 , 18.4 , 18.5 , 18.65, 18.7 , 19.  , 19.1 , 19.2 , 19.2 ,
        19.25, 19.3 , 19.4 , 19.45, 19.8 , 19.85, 19.9 , 20.1 , 21.2 ,
        21.5 , 21.7 , 22.  , 22.1 , 22.15, 22.2 , 22.3 , 22.45, 22.95,
        23.15, 23.6 , 23.9 , 24.1 , 24.15, 24.35, 24.4 , 24.6 , 24.9 ,
        25.  , 25.3 , 27.9 , 28.  , 28.6 , 28.85, 29.6 , 29.75, 29.95,
        30.85, 31.55, 32.25, 32.7 , 32.95, 35.  , 36.2 , 36.3 , 37.2 ,
        37.6 , 39.25, 43.3 , 43.9 , 47.5 , 50.  ]),
 array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 3, 1, 1,
        3, 1, 1, 1, 1, 2, 2, 5, 2, 1, 2, 2, 4, 1, 4, 1, 1, 1, 1, 2, 1, 1,
        2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1,
        2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 3, 3, 1, 1, 2, 1, 1, 2, 1, 8]))